In [11]:
from tkinter import YView, XView

import numpy as np
import pandas as pd

data = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

In [12]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [13]:
data['Age'].fillna(data['Age'].mean(),inplace=True)
data['Cabin'].fillna('N',inplace=True)
data['Embarked'].fillna('N',inplace=True)

/tmp/ipykernel_9907/3057721139.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(data['Age'].mean(),inplace=True)
/tmp/ipykernel_9907/3057721139.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', t

In [14]:
data.loc[data["Sex"] == "male", "Sex_encode"] = 0
data.loc[data["Sex"] == "female", "Sex_encode"] = 1

In [15]:
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩 수행.
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [16]:
Y = data['Survived']
X = data.drop('Survived', axis=1)


In [17]:
X = transform_features(X)

/tmp/ipykernel_9907/3807705002.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].mean(), inplace=True)
/tmp/ipykernel_9907/3807705002.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, Y, \
                                                  test_size=0.2, random_state=11)

In [19]:
def batch_sampler(data, target, batch_size):
    tot = len(data)
    total_iter = tot // batch_size
    for i in range(total_iter):
        yield (data[i * batch_size:(i + 1) * batch_size],
               target[i * batch_size:(i + 1) * batch_size])


sampler = batch_sampler(data=X_train, target=y_train, batch_size=32)
L = len(X_train) // 32
for i in range(L):
    next(sampler)
print(X_train)

     Pclass  Sex   Age  SibSp  Parch      Fare  Cabin  Embarked  Sex_encode
333       3    1  16.0      2      0   18.0000      7         3         0.0
662       1    1  47.0      0      0   25.5875      4         3         0.0
382       3    1  32.0      0      0    7.9250      7         3         0.0
331       1    1  45.5      0      0   28.5000      2         3         0.0
149       2    1  42.0      0      0   13.0000      7         3         0.0
..      ...  ...   ...    ...    ...       ...    ...       ...         ...
269       1    0  35.0      0      0  135.6333      2         3         1.0
337       1    0  41.0      0      0  134.5000      4         0         1.0
91        3    1  20.0      0      0    7.8542      7         3         0.0
80        3    1  22.0      0      0    9.0000      7         3         0.0
703       3    1  25.0      0      0    7.7417      7         2         0.0

[712 rows x 9 columns]


In [35]:
import numpy as np
from tqdm import tqdm
class LogisticRegression:
    def __init__(self, learning_rate=0.01, epochs=400, n_features=9):
        # 초기 가중치를 무작위로 설정
        self.weights = np.random.randn(n_features)
        self.b = np.random.randn()
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.loss_history = []

    def sigmoid(self, z):
        """Sigmoid 활성화 함수"""
        return 1.0 / (1 + np.exp(-z))

    def compute_loss(self, preds, target):
        """교차 엔트로피 손실 계산"""
        # 로그의 안정성을 위해 값들을 클리핑
        preds = np.clip(preds, 1e-9, 1 - 1e-9)  # 0과 1의 범위를 벗어나지 않도록 함
        loss = -((target * np.log(preds)) + (1 - target) * np.log(1 - preds)).mean()
        return loss


    def fit(self, X, y, batch_sampler):
        """모델 훈련 함수"""
        for epoch in tqdm(range(self.epochs)):
            sampler = batch_sampler(data=X, target=y, batch_size=4)
            batch_loss = []
    
            for X_batch, Y_batch in sampler:
                # 순방향 전파
                preds = self.sigmoid(np.dot(X_batch, self.weights) + self.b)
    
                # 손실 계산
                loss = self.compute_loss(preds, Y_batch)
                batch_loss.append(loss)
    
                # 그래디언트 계산
                dloss_dz = preds - Y_batch
                weights_deriv = np.dot(X_batch.T, dloss_dz) / X_batch.shape[0]
                b_deriv = dloss_dz.mean()
    
                # 가중치 및 바이어스 업데이트
                self.weights -= self.learning_rate * weights_deriv
                self.b -= self.learning_rate * b_deriv
    
            # Epoch별 손실 저장
            epoch_loss = np.mean(batch_loss)
            self.loss_history.append(epoch_loss)
            
            # 에포크마다 손실 출력
            print(f"Epoch {epoch + 1}/{self.epochs} - Loss: {epoch_loss:.4f}")


    def predict(self, X):
        """모델 예측 함수"""
        linear_comb = np.dot(X, self.weights) + self.b
        return self.sigmoid(linear_comb) > 0.55

    def accuracy(self, X, y):
        """정확도 계산 함수"""
        preds = self.predict(X)
        return (preds == y).mean() * 100




In [36]:
model = LogisticRegression(learning_rate=0.005, epochs=150, n_features=9)
model.fit(X_train, y_train, batch_sampler=batch_sampler)

  1%|▏         | 2/150 [00:00<00:09, 15.61it/s]

Epoch 1/150 - Loss: 2.2475
Epoch 2/150 - Loss: 1.7254
Epoch 3/150 - Loss: 1.6947
Epoch 4/150 - Loss: 1.6814


  4%|▍         | 6/150 [00:00<00:09, 15.72it/s]

Epoch 5/150 - Loss: 1.6694
Epoch 6/150 - Loss: 1.6585


  7%|▋         | 10/150 [00:00<00:08, 15.88it/s]

...

Epoch 140/150 - Loss: 1.5435
Epoch 141/150 - Loss: 1.5435
Epoch 142/150 - Loss: 1.5435
Epoch 143/150 - Loss: 1.5435


 97%|█████████▋| 146/150 [00:09<00:00, 15.99it/s]

Epoch 144/150 - Loss: 1.5435
Epoch 145/150 - Loss: 1.5434
Epoch 146/150 - Loss: 1.5434
Epoch 147/150 - Loss: 1.5434


100%|██████████| 150/150 [00:09<00:00, 15.89it/s]

Epoch 148/150 - Loss: 1.5434
Epoch 149/150 - Loss: 1.5433
Epoch 150/150 - Loss: 1.5433


In [37]:
# X_test에 대한 예측값 생성
predictions = model.predict(X_test)

# 정확도 계산
def calculate_accuracy(predictions, y_test):
    """예측값과 실제값을 비교하여 정확도를 계산"""
    correct_predictions = (predictions == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100
    return accuracy

# y_test와 비교하여 정확도 계산
accuracy = calculate_accuracy(predictions, y_test)
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 84.36%


In [38]:
model.accuracy(X_test, y_test)

np.float64(84.35754189944134)